## SPH Modelling

In [6]:
import numpy as np
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import cv2

# Constants
N     = 1000                  # Number of particles
t     = 0                     # Start time
tEnd  = 10                    # End time
dt    = 0.01                  # Timestep
M     = 1.0                   # Central star mass
h     = 0.1                   # Smoothing length
k     = 0.1                   # Equation of state constant
n     = 1                     # Polytropic index
nu    = 0.1                   # Damping (viscosity)
m     = M / N                 # Single particle mass
lmbda = 0.5                   # Gravity constant
Nt    = int(np.ceil(tEnd / dt)) # Number of timesteps
save_interval = 1            # Save plot every steps

# Initial Conditions: Disk-like distribution
np.random.seed(42)
phi = 2 * np.pi * np.random.rand(N)
r = 0.5 + 1.5 * np.random.rand(N)
z = 0.1 * np.random.randn(N)

# Convert to Cartesian coordinates
x = r * np.cos(phi)
y = r * np.sin(phi)
pos = np.vstack((x, y, z)).T

# Circular velocity to balance gravity
v_circ = np.sqrt(lmbda * M / r)
vel = np.vstack((-v_circ * np.sin(phi), v_circ * np.cos(phi), 0.01 * np.random.randn(N))).T

# Kernel function
def kernel(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)
    w = (1.0 / (h * np.sqrt(np.pi)))**3 * np.exp(-r**2 / h**2)
    return w

# Gradient of the kernel
def gradKernel(x, y, z, h):
    r = np.sqrt(x**2 + y**2 + z**2)
    n = -2 * np.exp(-r**2 / h**2) / h**5 / (np.pi)**(3/2)
    return n * x, n * y, n * z

# Magnitude between particles
def magnitude(ri, rj):
    M = ri.shape[0]
    N = rj.shape[0]
    dx = ri[:, 0].reshape((M, 1)) - rj[:, 0].reshape((N, 1)).T
    dy = ri[:, 1].reshape((M, 1)) - rj[:, 1].reshape((N, 1)).T
    dz = ri[:, 2].reshape((M, 1)) - rj[:, 2].reshape((N, 1)).T
    return dx, dy, dz

# Density calculation
def density(r, pos, m, h):
    dx, dy, dz = magnitude(r, pos)
    return np.sum(m * kernel(dx, dy, dz, h), axis=1).reshape(-1, 1)

# Pressure calculation
def pressure(rho, k, n):
    return k * rho**(1 + 1/n)

# Acceleration calculation
def acceleration(pos, vel, m, h, k, n, lmbda, nu):
    rho = density(pos, pos, m, h)
    P = pressure(rho, k, n)
    dx, dy, dz = magnitude(pos, pos)
    dWx, dWy, dWz = gradKernel(dx, dy, dz, h)
    ax = -np.sum(m * (P / rho**2 + P.T / rho.T**2) * dWx, axis=1).reshape(-1, 1)
    ay = -np.sum(m * (P / rho**2 + P.T / rho.T**2) * dWy, axis=1).reshape(-1, 1)
    az = -np.sum(m * (P / rho**2 + P.T / rho.T**2) * dWz, axis=1).reshape(-1, 1)
    a = np.hstack((ax, ay, az))
    a -= lmbda * pos / (np.linalg.norm(pos, axis=1, keepdims=True) + 1e-3)**3
    return a - nu * vel

# Create output directory
if not os.path.exists('output_3D_pdisk'):
    os.mkdir('output_3D_pdisk')

# Main loop
for i in tqdm.tqdm(range(Nt)):
    acc = acceleration(pos, vel, m, h, k, n, lmbda, nu)
    vel += acc * dt
    pos += vel * dt
    rho = density(pos, pos, m, h)

    if i % save_interval == 0:
        # 3D Plotting
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')
        cval = np.minimum((rho - 3) / 3, 1).flatten()
        ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], c=cval, cmap=plt.cm.inferno, s=5, alpha=0.75)
        ax.scatter(0, 0, 0, color='yellow', s=50, marker='*')  # Central star
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)
        ax.set_zlim(-0.5, 0.5)
        ax.view_init(elev=30, azim=45)  # Angled view
        plt.savefig(f'output_3D_pdisk/{i}.png')
        plt.close()

# Animation
import re
img_array = []
imgs_list = glob.glob('output_3D_pdisk/*.png')
imgs_sorted = sorted(imgs_list, key=lambda x: int(re.search(r'\d+', os.path.basename(x)).group()))

for filename in imgs_sorted:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter('protoplanetary_disk_3.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, size)
for img in img_array:
    out.write(img)
out.release()


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:30<00:00, 11.08it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
